# Evaluation of study results

In [ ]:
import numpy as np
import geopandas as gpd
import folium
import pandas as pd
from branca.colormap import linear

In [ ]:
MAX_PLZ_STELLEN = 3

In [ ]:
data = gpd.read_file("data/Umfragedaten.csv")
plz_gebiete = gpd.read_file("data/georef-germany-postleitzahl.geojson")

In [ ]:
shortened_plz = plz_gebiete["name"].apply(lambda x: x[:MAX_PLZ_STELLEN])
plz_gebiete["shortened_plz"] = shortened_plz
shortened_plz = data["Was ist Ihre Postleitzahl?"].apply(lambda x: x[:MAX_PLZ_STELLEN])
data["shortened_plz"] = shortened_plz

In [ ]:
short_plzs = []
combined_polygons = []
for _, short_plz in enumerate(plz_gebiete["shortened_plz"].unique()):
    all_areas = plz_gebiete.query("`shortened_plz` == @short_plz")
    polygons = all_areas["geometry"].union_all()
    short_plzs.append(short_plz)
    combined_polygons.append(polygons)

new_plz_gebiete = gpd.GeoDataFrame.from_dict({"shortened_plz": short_plzs, "geometry": combined_polygons}, crs="EPSG:4326")

In [ ]:
merged_data = pd.merge(data, new_plz_gebiete, on="shortened_plz")
merged_geo_df = gpd.GeoDataFrame(merged_data, geometry="geometry")
prevelance = merged_geo_df["shortened_plz"].value_counts()

In [ ]:
merged_geo_df["shortened_plz"]

In [ ]:
merged_geo_df["prevelance"] = merged_geo_df["shortened_plz"].apply(lambda x: prevelance[f"{x}"])

In [ ]:
m = folium.Map(location=[52, 13], zoom_start=6, tiles="Cartodb Positron", bbox=[10,90])

In [ ]:
# m.add_child(folium.ColorMap(merged_geo_df["prevelance"].min(), merged_geo_df["prevelance"].max()))

In [ ]:
colormap = linear.GnBu_09.scale(merged_geo_df["prevelance"].min() - 10, merged_geo_df["prevelance"].max())

In [ ]:
folium.GeoJson(merged_geo_df, style_function=lambda row: {
    "fillColor": colormap(row["properties"]["prevelance"]),
    "weight": 0,
    "fillOpacity": 5
}).add_to(m)

In [ ]:
hex_colors = []
max_val = merged_geo_df["prevelance"].max()
min_val = merged_geo_df["prevelance"].min()
for x in range(max_val, min_val - 1, -1):
    hex_colors.append(colormap(x))
html_element = f"""
<div style="top: 40px; right: 185px; z-index: 1000; position: fixed;">{max_val}</div>
<div style="
    position: fixed;
    top: 50px; right: 150px; width: 30px; height: 200px;
    background: linear-gradient({', '.join(hex_colors)});
    z-index: 1000;
"></div>
<div style="top: 240px; right: 185px; z-index: 1000; position: fixed;">{min_val}</div>
<div style="top: 140px; right: 72px; transform: rotate(90deg); z-index: 1000; position: fixed">Anzahl der Befragten</div>
<img src="North_Pointer_minus_6deg.png" style="top: 50px; left:50px; position: fixed; z-index: 1000; height: 75; width: 75px;">
"""
print(html_element)
m.get_root().html.add_child(folium.Element(html_element))

In [ ]:
m

In [ ]:
m.save("test.html")